### 1. Settings

In [40]:
#####################################
##########  DEPENDECIES ############
#####################################

import os
import pickle
from tqdm import tqdm # type: ignore
from datetime import date

import evaluate

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import utils.prep as pr
import utils.eval as ev
import utils.inference as infer
from utils.sampling import create_splits, prep_cv_validation
from utils.training import cv_cluster_set, cv_training_epochs_sets, test_cluster_set
from utils.training import results_dict_todf, cv_step_2, full_step_2, test_training_epochs_sets
from utils.inference import meta_predict, create_ensemble_map

tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
#####################################
############  CONSTANTS #############
#####################################
RS = 42

MODEL = "CodeT5"
BATCH_SIZE = 16
DECODER_LENGTH = 20
ENCODER_LENGTH = 30
DATE_STR = 20240721
model_name="Salesforce/codet5-base-multi-sum"

FULL_TRAIN_ARGS = {
    "BATCH_SIZE": BATCH_SIZE,
    "DECODER_LENGTH": DECODER_LENGTH,
    "ENCODER_LENGTH": ENCODER_LENGTH,
    "MODEL": MODEL,
    "SEQ_TRAINER_ARGS": {
        "overwrite_output_dir": True,
        "num_train_epochs": [0, 1, 5, 8],
        "do_train": True,
        "do_eval": True,
        "per_device_train_batch_size": 4,
        "per_device_eval_batch_size": 4,
        "learning_rate": 6e-6,
        "warmup_steps": 500,
        "weight_decay": 0.1,
        "label_smoothing_factor": 0.1,
        "predict_with_generate": True,
        "logging_steps": 100,
        "save_total_limit": 1,
        "save_strategy": "no",
        "logging_strategy": "epoch",
        "evaluation_strategy": "epoch",
        "load_best_model_at_end": False,
        "output_dir" : 'reports/results',
        "logging_dir" : "reports/logs",
    },
}

tokenizer = AutoTokenizer.from_pretrained(model_name, skip_special_tokens=False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
rouge = evaluate.load('rouge')

### 2. Conala data. Preprocessing. 

In [41]:
experiment_config = {
    "DATE_STR" : "20240721",
    "RS" : 42,
    "DRIFT_TYPE" : "sudden",
    "NFOLD" : 3,
    "FULL_TRAIN_ARGS" : FULL_TRAIN_ARGS,
    "MODEL_NAME" : model_name,
    "CLUSTER_EPOCHS" : 2,
}
experiment_config["ANALYSIS_POSTFIX"] = f"mined_{experiment_config['DRIFT_TYPE']}_{str(date.today())}"

In [3]:
sampling_dict = create_splits(experiment_config=experiment_config, tokenizer=tokenizer, test=False)
train_dataset, test_data, test_df = sampling_dict["train_data"], sampling_dict["test_data"], sampling_dict["test_df"]

splits, questions_list = prep_cv_validation(train_dataset=train_dataset, 
                            experiment_config=experiment_config)

Train Data:  (7942, 11)
Test Data:  (2058, 11)
Train Data: Cluster cluster
2    3632
3    2204
1    1672
0     414
4      20
Name: count, dtype: int64
Test Data: Cluster cluster
4    1980
3      39
2      25
1      12
0       2
Name: count, dtype: int64


Filter:   0%|          | 0/2058 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2058 [00:00<?, ? examples/s]

Map:   0%|          | 0/2058 [00:00<?, ? examples/s]

Fold 0
Fold 1
Fold 2


In [4]:
fold_results = cv_training_epochs_sets(experiment_config=experiment_config,
                            splits=splits,
                            questions_list=questions_list,
                            train_dataset=train_dataset,
                            tokenizer=tokenizer)

Fold 0


Filter:   0%|          | 0/7942 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7942 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5298 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5298 [00:00<?, ? examples/s]

...

LOADING MODEL ./tmp/
cuda


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,4.678800,4.577013,0.260800,0.072200,0.232000,0.232000,14.021900,0.024500,1.000000,1.148500,29033,25280
2,4.574500,4.520506,0.269400,0.073800,0.238000,0.238100,13.941500,0.024900,1.000000,1.138200,28773,25280
3,4.469600,4.501167,0.273700,0.075300,0.241800,0.241700,14.143000,0.025200,1.000000,1.157100,29252,25280
4,4.422700,4.495306,0.273600,0.075100,0.241300,0.241400,14.093200,0.024500,1.000000,1.151700,29114,25280


TRAINING EPOCH SET 8
TRAINING EPOCHS 3
LOADING MODEL ./tmp/
cuda


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,4.241100,4.499412,0.268200,0.073500,0.237200,0.237200,14.003400,0.023800,1.000000,1.137200,28748,25280
2,4.208900,4.486858,0.271000,0.073900,0.239200,0.239200,13.969400,0.024700,1.000000,1.136200,28722,25280
3,4.188600,4.485295,0.270300,0.074000,0.238200,0.238300,14.064500,0.024500,1.000000,1.145100,28949,25280


In [5]:
for cluster_idx in [1, 4, 3]:
    fold_results = cv_cluster_set(experiment_config=experiment_config,
                                    splits=splits,
                                    questions_list=questions_list,
                                    train_dataset=train_dataset,
                                    tokenizer=tokenizer,
                                    fold_results=fold_results,
                                    cluster_id=cluster_idx)

cv_df = results_dict_todf(fold_results)

########## SAVE THE FILE

with open(f'reports/results/cv_result_{experiment_config["ANALYSIS_POSTFIX"]}.pickle', 'wb') as handle:
    pickle.dump(cv_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

Fold 0


Filter:   0%|          | 0/7942 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7942 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5291 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5291 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/2651 [00:00<?, ? examples/s]

TRAINING CLUSTER SET 3 FOR EPOCHS2


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


LOADING MODEL Salesforce/codet5-base-multi-sum
cuda


NameError: name 'ANALYSIS_POSTFIX' is not defined

In [18]:
fold_results['cluster_1'][0]['rouge'].mean()

0.2592078728820197

In [7]:

fold_results.keys()

dict_keys([0, 1, 5, 8, 'cluster_1', 'cluster_4', 'cluster_3'])

In [8]:
print("Mean")
print(cv_df.groupby(["model_set"])["rouge"].mean())

print("STD")
print(cv_df.groupby("model_set")["rouge"].std())

Mean
model_set
0            0.113737
1            0.247452
5            0.273484
8            0.273170
cluster_1    0.162001
cluster_3    0.160304
cluster_4    0.156176
Name: rouge, dtype: float64
STD
model_set
0            0.124526
1            0.157347
5            0.158682
8            0.158973
cluster_1    0.153707
cluster_3    0.151417
cluster_4    0.149950
Name: rouge, dtype: float64


In [20]:
########## LOAD CV RESULTS

import pickle
with open(f'reports/results/cv_result_{experiment_config["ANALYSIS_POSTFIX"]}.pickle', 'rb') as handle:
    cv_df = pickle.load(handle)

### Step 2. Learn performance

In [21]:
cv_df, model_results = cv_step_2(experiment_config=experiment_config, cv_df=cv_df)

with open(f'reports/results/s2_model_results_{experiment_config["ANALYSIS_POSTFIX"]}.pickle', 'wb') as handle:
    pickle.dump(model_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'reports/results/cd_df_with_s2_{experiment_config["ANALYSIS_POSTFIX"]}.pickle', 'wb') as handle:
    pickle.dump(cv_df, handle, protocol=pickle.HIGHEST_PROTOCOL)


0
lr
svm
lgbm
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.262464 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 14595
[LightGBM] [Info] Number of data points in the train set: 37086, number of used features: 1062
[LightGBM] [Info] Start training from score 0.178388
catboost
Learning rate set to 0.072463
0:	learn: 0.1572665	total: 83.2ms	remaining: 1m 23s
1:	learn: 0.1550724	total: 95.6ms	remaining: 47.7s
2:	learn: 0.1531492	total: 105ms	remaining: 35s
3:	learn: 0.1514378	total: 114ms	remaining: 28.3s
4:	learn: 0.1499549	total: 123ms	remaining: 24.4s
5:	learn: 0.1486640	total: 130ms	remaining: 21.5s
6:	learn: 0.1475208	total: 139ms	remaining: 19.7s
7:	learn: 0.1464843	total: 147ms	remaining: 18.2s
8:	learn: 0.1456481	total: 155ms	remaining: 17s
9:	learn: 0.1449033	total: 162ms	remaining: 16.1s
10:	learn: 0.1442216	total: 170ms	remai

In [ ]:

print("Mean")
print(cv_df.groupby(["model_set"])["rouge"].mean())

print("STD")
print(cv_df.groupby("model_set")["rouge"].std())

### TO SAVE THE VECTORIZER AND STEP 2 MODELS

with open(f'reports/results/cd_df_with_s2_{experiment_config["ANALYSIS_POSTFIX"]}.pickle', 'rb') as handle:
    cv_df = pickle.load(handle)

In [23]:
full_step_2(cv_df=cv_df, 
            experiment_config=experiment_config)

lr
svm
lgbm
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.058661 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 21184
[LightGBM] [Info] Number of data points in the train set: 55594, number of used features: 1420
[LightGBM] [Info] Start training from score 0.198046
catboost
Learning rate set to 0.077249
0:	learn: 0.1609318	total: 23.3ms	remaining: 23.3s
1:	learn: 0.1594933	total: 37.1ms	remaining: 18.5s
2:	learn: 0.1582267	total: 48.6ms	remaining: 16.1s
3:	learn: 0.1571694	total: 59.6ms	remaining: 14.8s
4:	learn: 0.1562638	total: 71.6ms	remaining: 14.3s
5:	learn: 0.1554548	total: 81.3ms	remaining: 13.5s
6:	learn: 0.1547782	total: 91.5ms	remaining: 13s
7:	learn: 0.1541655	total: 103ms	remaining: 12.8s
8:	learn: 0.1536466	total: 114ms	remaining: 12.5s
9:	learn: 0.1532312	total: 124ms	remaining: 12.2s
10:	learn: 0.1528646	total: 135ms	r

# TEST

In [24]:
sampling_dict = create_splits(experiment_config=experiment_config, tokenizer=tokenizer, test=True)
train_dataset, test_data, test_df = sampling_dict["train_data"], sampling_dict["test_data"], sampling_dict["test_df"]

splits, questions_list = prep_cv_validation(train_dataset=train_dataset, 
                            experiment_config=experiment_config)

Train Data:  (7942, 11)
Test Data:  (2058, 11)
Train Data: Cluster cluster
2    3632
3    2204
1    1672
0     414
4      20
Name: count, dtype: int64
Test Data: Cluster cluster
4    1980
3      39
2      25
1      12
0       2
Name: count, dtype: int64


Filter:   0%|          | 0/7942 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7942 [00:00<?, ? examples/s]

Map:   0%|          | 0/7942 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2058 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2058 [00:00<?, ? examples/s]

Map:   0%|          | 0/2058 [00:00<?, ? examples/s]

Fold 0
Fold 1
Fold 2


In [26]:
with open(f"reports/results/cd_df_with_s2_{experiment_config['ANALYSIS_POSTFIX']}.pickle", "rb") as handle:
    cv_resutls = pickle.load(handle)

base_models_list = list(cv_resutls.model_set.unique())
base_models_list.pop(-1)

'ensemble'

In [29]:
meta_preds_df = meta_predict(experiment_config=experiment_config, 
                    test_df=test_df,
                    base_models_names=base_models_list,
                    t_models=["svm", "catboost"])

########## SAVE THE FILE

with open(f'reports/results/test_results_s2_{experiment_config["ANALYSIS_POSTFIX"]}.pickle', 'wb') as handle:
    pickle.dump(meta_preds_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

svm
catboost


In [42]:
with open(f'reports/results/test_results_s2_{experiment_config["ANALYSIS_POSTFIX"]}.pickle', 'rb') as handle:
    meta_preds_df = pickle.load(handle)

FileNotFoundError: [Errno 2] No such file or directory: 'reports/results/test_results_s2_mined_sudden_2024-08-10.pickle'

In [62]:
with open(f'reports/results/test_results_s2_mined_sudden_2024-08-09.pickle', 'rb') as handle:
    meta_preds_df = pickle.load(handle)

In [63]:
meta_preds_df.groupby("model_set").svm_preds.mean()

model_set
0            0.121347
1            0.253740
5            0.280409
8            0.276581
cluster_1    0.161217
cluster_3    0.159681
cluster_4    0.157979
Name: svm_preds, dtype: float64

In [64]:
optimal_ensemble_map = create_ensemble_map(meta_preds_df=meta_preds_df, 
                                           t_model_name="catboost")

In [68]:
test_result_df = test_training_epochs_sets(experiment_config=experiment_config,
                            test_df=test_df,
                            test_data=test_data,
                            train_data=train_dataset,
                            tokenizer=tokenizer)

TRAINING EPOCH SET 0
TRAINING EPOCHS 0
cuda


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


TRAINING EPOCH SET 1
TRAINING EPOCHS 1
cuda


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,5.040200,4.602952,0.264300,0.065200,0.230100,0.230400,14.460600,0.019500,1.000000,1.187800,23603,19871


OutOfMemoryError: CUDA out of memory. Tried to allocate 182.00 MiB. GPU 0 has a total capacity of 44.46 GiB of which 36.19 MiB is free. Process 1549668 has 4.78 GiB memory in use. Including non-PyTorch memory, this process has 36.85 GiB memory in use. Process 1463273 has 2.78 GiB memory in use. Of the allocated memory 32.09 GiB is allocated by PyTorch, and 4.44 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [61]:
test_result_df.columns

Index(['question_id', 'parent_answer_post_id', 'prob', 'input_sequence',
       'output_sequence', 'id', 'snippet_len', 'intent_len', 'snippet_token_n',
       'intent_token_n', 'cluster', 'input_ids', 'attention_mask', 'labels',
       'prediction', 'rouge', 'epoch_set', 'cluster_1'],
      dtype='object')

In [59]:
test_result_dict.model_set.unique()

AttributeError: 'DataFrame' object has no attribute 'model_set'

In [55]:
for cluster_idx in [1, 4, 3]:
    test_result_dict = test_cluster_set(experiment_config=experiment_config,
                                    test_df=test_df,
                                    test_data=test_data,
                                    tokenizer=tokenizer,
                                    results=test_result_df,
                                    cluster_id=cluster_idx)

test_result_df = results_dict_todf(test_result_dict)

########## SAVE THE FILE

with open(f'reports/results/test_results_df_{experiment_config["ANALYSIS_POSTFIX"]}.pickle', 'wb') as handle:
    pickle.dump(test_result_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

Filter:   0%|          | 0/7942 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7942 [00:00<?, ? examples/s]

Map:   0%|          | 0/7942 [00:00<?, ? examples/s]

TRAINING CLUSTER SET 1 FOR EPOCHS2
LOADING MODEL Salesforce/codet5-base-multi-sum
cuda


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,4.951700,4.590503,0.256500,0.063500,0.225100,0.225400,14.527200,0.021300,1.000000,1.197300,23792,19871
2,4.569700,4.547361,0.263000,0.067100,0.231500,0.231800,14.380000,0.023000,1.000000,1.178600,23420,19871


ValueError: Columns must be same length as key

In [ ]:
### ENSEMBLE COMPUTE
test_result_df = ensemble_compute(test_result_df=test_result_df,
                                  optimal_ensemble_map=optimal_ensemble_map)

In [ ]:
########## ROUGE PER SETTING

print("Mean")
print(test_result_df.groupby("epoch_set")["rouge"].mean())

print("STD")
print(test_result_df.groupby("epoch_set")["rouge"].std())

In [ ]:
test_result_df.opt_es_id.value_counts()